# Issue a Credential using BIP 322 Signatures

This notebook outlines how a Bitcoin address can be used to sign a [W3C Verifiable Credential](https://www.w3.org/TR/vc-data-model/) using BIP 322 signatures following the [W3C Data Integrity Specification](https://w3c.github.io/vc-data-integrity).

A detailed walkthrough of how BIP 322 signatures work can be found in [this notebook](../BIP0322_signing.ipynb).

## Notebook Author

* Will Abramson
* [Legendary Requirements](http://legreq.com/)
* Contact: will@legreq.com

## Acknowledgements

This work was funded by Ryan Grant and Digital Contract Design. Thanks also go to Joe Andrieu, Kalle Alm, Pieter Wuille and Jimmy Song for engaging with and supporting various aspects of this work.

## Imports

In [1]:
cd ..

/home/will/work/LegendaryRequirements/clients/dcd/bip0322-sigs


In [13]:
import copy
import datetime

In [61]:
# buidl-python dependencies
from buidl.ecc import PrivateKey
from buidl.helper import sha256, bytes_to_str, str_to_bytes

In [4]:
# BIP322 Signature Implementation
from src.message import sign_message, MessageSignatureFormat

In [5]:
# Contains URDNA2015 cannonicalization library from Digital Bazaar - https://github.com/digitalbazaar/pyld
from pyld import jsonld

## Define Verification Method

Before a Data Integrity proof can be created for a VC, the issuer have a verificationMethod that is identifiable through a URL, typically a Decentralized Identifier. This is then included in the VC such that a verifier can dereference the URL to retrieve the public material necessary to verify the proof. Typically this is a cryptographic public key and associated alogorithm, however in the case of BIP 322 this will be a bitcoin address that should be used with the BIP 322 verification mechanism.

### Steps

1. Create secp256K1 Private/Public Key pair
2. Get valid bitcoin address associated with the public key
3. Include bitcoin address as a BIP322 verificationMethod in a DID document

### 1. Create secp256K1 Private/Public Key pair

In [6]:
private_key_wif = "L4qmTC36US6tngrqbrW6vffZbxUVSfipQqXxdJw8S5HqyxNebaMP"
private_key = PrivateKey.parse(private_key_wif)
public_key = private_key.point

print("Private Key : ", private_key)
print("Public Key : ", public_key)

Private Key :  <buidl.pecc.PrivateKey object at 0x7f18d5602d90>
Public Key :  S256Point(03c8822574d7c7055ade7c7dd598dd5af47e72354b56e3fa1e2c2c3cf025f6cf31)


### 2. Get valid bitcoin address associated with the public key

Note: there are many valid bitcoin addresses that can be used: p2pkh, p2wpkh, p2tr, p2sh. These can encode complex verification rules including t-of-m multisignatures. This is one of the advantages of BIP322 over traditional public key signaturte schemes. This is discussed on more detail [here](https://github.com/WebOfTrustInfo/rwot11-the-hague/blob/master/advance-readings/bip322-signature-suite.md).

For this example we will use a simple p2wpkh, such that a valid BIP322 signature can be created with a signature from the private key created in step 1 over data in a specific format specified by the BIP322 specification.

In [7]:
address = private_key.point.p2wpkh_address()
print("Bitcoin address : ", address)

Bitcoin address :  bc1qz52z3pe9fg3qxv9n6yhxgj7rcn8wsvpq56v9ck


### 3. Include bitcoin address as a BIP322 verificationMethod in a DID document

A verificationMethod is defined in the [W3C DID specification](https://www.w3.org/TR/did-core/#verification-methods). It needs to include:

* id: The value of the id property for a verification method MUST be a string that conforms to the rules in Section 3.2 DID URL Syntax.
* type: The value of the type property MUST be a string that references exactly one verification method type. In order to maximize global interoperability, the verification method type SHOULD be registered in the DID Specification Registries [DID-SPEC-REGISTRIES]. 
* controller: The value of the controller property MUST be a string that conforms to the rules in 3.1 DID Syntax. 

TODO: A verificationMethod for BIP322 needs to be defined as part of a BIP322 Signature suite. 
**ISSUE:** The Data Integrity specification states that it:
> limits the number of formats for expressing verification material in a controller document. The fewer formats that implementers have to implement, the more likely it will be that they will support all of them.

Currently the spec only supports publicKeyJWK and publicKeyMultibase. We will need to extend this to include something like `address`, to hold the bitcoin address.

Note: Once a verificationMethod for BIP322 has been defined, it should be possible for most DID documents to add specific BIP322 as a verification method. In this example we extend a BTCR v0.1 document to include a WIP verificationMethod for BIP322. Resolution of this DID document from the DID is out of scope for this example.

In [8]:
# Example test net did:btcr (taken from 0.1 spec)
issuer_did_btcr = "did:btcr:xyv2-xzpq-q9wa-p7t"

In [9]:
# Currently resolves to this
did_doc = {
    "@context": ["https://w3id.org/did/v0.11", "https://w3id.org/btcr/v1"],
    "id": "did:btcr:xyv2-xzpq-q9wa-p7t",
    "verificationMethod": [
       {
            "id": "did:btcr:xyv2-xzpq-q9wa-p7t#satoshi",
            "controller": "did:btcr:xyv2-xzpq-q9wa-p7t",
            "type": "EcdsaSecp256k1VerificationKey2019",
            "publicKeyBase58": "owh12LKNuphe97teJTZKQTKNewSVTwjHcskPbq34epCY"
        },
        {
             "id": "did:btcr:xyv2-xzpq-q9wa-p7t#vckey-0",
             "controller": "did:btcr:xyv2-xzpq-q9wa-p7t",
             "type": "EcdsaSecp256k1VerificationKey2019",
             "publicKeyBase58": "owh12LKNuphe97teJTZKQTKNewSVTwjHcskPbq34epCY"
         },
    ##########################      EXAMPLE       #################################
        ### What are the resolution rules that parse the Bitcoin Tx at ref xyv2-xzpq-q9wa-p7t
        ### To populate this verificationMethod into the didDocument.
        ### E.g. scriptPubKey from UTX0 at index 1 in tx at ref xyv2-xzpq-q9wa-p7t
        {
             "id": "did:btcr2:xyv2-xzpq-q9wa-p7t#vm-1",
             "controller": "did:btcr:xyv2-xzpq-q9wa-p7t",
             "type": "BIP322VerificationAddress",
             "address": address
        }
    
    ###############################################################################
    ],
    "authentication": ["#satoshi"],
    "assertionMethod": ["#vckey-0","#vm-1"]
}


## Define a VC to be Signed

Issuer needs to decide:

* Contents of credential
* Schema of credential
* Key pair and associated identifier they will be signing with
* Identifier to use to the credential subject

In [10]:
unsigned_vc = {
  
  "@context": [
    "https://www.w3.org/2018/credentials/v1",
    "https://www.w3.org/2018/credentials/examples/v1"
  ],
  
  "id": "http://example.edu/credentials/1872",
  


  "type": ["VerifiableCredential", "AlumniCredential"],
  

  "issuer": issuer_did_btcr,
    

  "issuanceDate": "2010-01-01T19:23:24Z",
  


  "credentialSubject": {
    

    # TODO: How identify credential subject?
    "id": "did:example:ebfeb1f712ebc6f1c276e12ec21",
    "alumniOf": {
      "id": "did:example:c276e12ec21ebfeb1f712ebc6f1",
      "name": [{
        "value": "Example University",
        "lang": "en"
      }, {
        "value": "Exemple d'Université",
        "lang": "fr"
      }]
    }
  },
  


}

## Create Data Integrity Proof for a VC

### Algorithm Steps taken from the [specification](https://w3c.github.io/vc-data-integrity/#proof-algorithm)


1. Create a copy of document, hereafter referred to as output.
2. Generate a canonicalized document by canonicalizing document according to a canonicalization algorithm (e.g. the URDNA2015 [RDF-DATASET-C14N] algorithm).
3. Create a value tbs that represents the data to be signed, and set it to the result of running the Create Verify Hash Algorithm, passing the information in options.
4. Digitally sign tbs using the privateKey and the the digital proof algorithm. The resulting string is the proofValue.
6. Add a proof node to output containing a data integrity proof using the appropriate type and proofValue values as well as all of the data in the proof options (e.g. created, and if given, any additional proof options such as domain).
7. Return output as the signed data document.



## 1. Create a copy of document (VC), hereafter referred to as output.



In [91]:
output = copy.deepcopy(unsigned_vc)
print(output)

{'@context': ['https://www.w3.org/2018/credentials/v1', 'https://www.w3.org/2018/credentials/examples/v1'], 'id': 'http://example.edu/credentials/1872', 'type': ['VerifiableCredential', 'AlumniCredential'], 'issuer': 'did:btcr:xyv2-xzpq-q9wa-p7t', 'issuanceDate': '2010-01-01T19:23:24Z', 'credentialSubject': {'id': 'did:example:ebfeb1f712ebc6f1c276e12ec21', 'alumniOf': {'id': 'did:example:c276e12ec21ebfeb1f712ebc6f1', 'name': [{'value': 'Example University', 'lang': 'en'}, {'value': "Exemple d'Université", 'lang': 'fr'}]}}}


## 2. Canonicalize VC

Uses the URDNA2015 algorithm implemented in the JSONLD library

In [12]:
canonicalized_output = jsonld.normalize(
    output, {'algorithm': 'URDNA2015', 'format': 'application/n-quads'})
print(canonicalized_output)

<did:example:c276e12ec21ebfeb1f712ebc6f1> <http://schema.org/name> _:c14n0 .
<did:example:c276e12ec21ebfeb1f712ebc6f1> <http://schema.org/name> _:c14n1 .
<did:example:ebfeb1f712ebc6f1c276e12ec21> <http://schema.org/alumniOf> <did:example:c276e12ec21ebfeb1f712ebc6f1> .
<http://example.edu/credentials/1872> <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <https://example.org/examples#AlumniCredential> .
<http://example.edu/credentials/1872> <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <https://www.w3.org/2018/credentials#VerifiableCredential> .
<http://example.edu/credentials/1872> <https://www.w3.org/2018/credentials#credentialSubject> <did:example:ebfeb1f712ebc6f1c276e12ec21> .
<http://example.edu/credentials/1872> <https://www.w3.org/2018/credentials#issuanceDate> "2010-01-01T19:23:24Z"^^<http://www.w3.org/2001/XMLSchema#dateTime> .
<http://example.edu/credentials/1872> <https://www.w3.org/2018/credentials#issuer> <did:btcr:xyv2-xzpq-q9wa-p7t> .



## 3. Run Create Verify Hash Algorithm on canonicalized output

Note: Create Verify Hash Algorithm steps are defined in the [Data Integrity specification](https://w3c.github.io/vc-data-integrity/#create-verify-hash-algorithm)

Steps:

1. Let options be a copy of input options.
2. If the proofValue parameter, such as jws, exists in options, remove the entry.
3. If created does not exist in options, add an entry with a value that is an [ISO8601] combined date and time string containing the current date and time accurate to at least one second, in Universal Time Code format. For example: 2017-11-13T20:21:34Z.
4. Generate output by:
    * Creating a canonicalized options document by canonicalizing options according to the canonicalization algorithm (e.g. the URDNA2015 [RDF-DATASET-C14N] algorithm).
    * Hash canonicalized options document using the message digest algorithm (e.g. SHA-256) and set output to the result.
    * Hash canonicalized document using the message digest algorithm (e.g. SHA-256) and append it to output.



#### 1. Let options be a copy of input options

Note: it is not at all clear from the spec what input_options should be.Commented on issues already tracking this here - https://github.com/w3c/vc-data-integrity/issues/16

For now I will be using the proof options that define how the proof will be created.

In [36]:
## THIS DOES NOT CURRENT WORK WITH CANNONICALIZATION
## The Context does not resolve

options = {
    ## This context URL is incorrect
  "@context": ["https://w3id.org/security/data-integrity/v1"],
  # I believe this is the proposed new format. Only one type - DataIntegritySignature - with many cryptosuites.  
  "type": "DataIntegritySignature",
  "cryptosuite": "bip322-2022",
  "created": str(datetime.datetime.now()),
  # Identifies the BIP322 verificationMethod in the issuers DID document
  "verificationMethod": issuer_did_btcr + '#vm-1',
  "proofPurpose": "assertionMethod",
#### NOTE: no proofValue as the proof has not been created yet
#   "proofValue": "z2rb7doJxczUFBTdV5F5pehtbUXPDUgKVugZZ99jniVXCUpojJ9PqLYV
#                  evMeB1gCyJ4HqpnTyQwaoRPWaD3afEZboXCBTdV5F5pehtbUXPDUgKVugUpoj"
}

In [45]:
options = {
  "@context": [
    ## If not using
    "https://w3id.org/security/suites/ed25519-2020/v1"
  ],
  "proof": {
     ## This should be something like BIP322Signature2022
    ## BUT No Context defined for it
    "type": "Ed25519Signature2020",
    "created": str(datetime.datetime.now()),
    "verificationMethod": issuer_did_btcr + '#vm-1',
    "proofPurpose": "assertionMethod"
  }
}

In [46]:
print(options)

{'@context': ['https://w3id.org/security/suites/ed25519-2020/v1'], 'proof': {'type': 'Ed25519Signature2020', 'created': '2022-08-17 13:58:30.266554', 'verificationMethod': 'did:btcr:xyv2-xzpq-q9wa-p7t#vm-1', 'proofPurpose': 'assertionMethod'}}


#### 2. If proofValue parameter exists in options, remove

Note: it does not exist in this example

#### 3 If created does not exists in option, dd an entry with a value that is an [ISO8601] combined date and time string containing the current date and time accurate to at least one second, in Universal Time Code format.

Options already contains this

#### 4. Generate output by:


####     4.1. Creating a canonicalized options document by canonicalizing options according to the canonicalization algorithm (e.g. the URDNA2015 [RDF-DATASET-C14N] algorithm).

In [83]:
canonicalized_options = jsonld.normalize(
    options, {'algorithm': 'URDNA2015', 'format': 'application/n-quads'})
print(canonicalized_options)

_:c14n1 <http://purl.org/dc/terms/created> "2022-08-17 13:58:30.266554"^^<http://www.w3.org/2001/XMLSchema#dateTime> _:c14n0 .
_:c14n1 <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <https://w3id.org/security#Ed25519Signature2020> _:c14n0 .
_:c14n1 <https://w3id.org/security#proofPurpose> <https://w3id.org/security#assertionMethod> _:c14n0 .
_:c14n1 <https://w3id.org/security#verificationMethod> <did:btcr:xyv2-xzpq-q9wa-p7t#vm-1> _:c14n0 .
_:c14n2 <https://w3id.org/security#proof> _:c14n0 .



#### 4.2 Hash canonicalized options document using the message digest algorithm (e.g. SHA-256) and set output to the result.

In [86]:
tst = str_to_bytes(canonicalized_options)
bytes_to_str(tst)

'_:c14n1 <http://purl.org/dc/terms/created> "2022-08-17 13:58:30.266554"^^<http://www.w3.org/2001/XMLSchema#dateTime> _:c14n0 .\n_:c14n1 <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <https://w3id.org/security#Ed25519Signature2020> _:c14n0 .\n_:c14n1 <https://w3id.org/security#proofPurpose> <https://w3id.org/security#assertionMethod> _:c14n0 .\n_:c14n1 <https://w3id.org/security#verificationMethod> <did:btcr:xyv2-xzpq-q9wa-p7t#vm-1> _:c14n0 .\n_:c14n2 <https://w3id.org/security#proof> _:c14n0 .\n'

In [69]:
result = sha256(str_to_bytes(canonicalized_options, 'utf-8'))
result

b'\x82\xb4\x93\x9e\xa9Mj\xf0~L)\xbe\xc5\xfd.F\x85\x1e\xa7\x97a\xafc\xbd\x10\x1b\x83\xfa\xd2u\x14\xeb'

#### 4.3 Hash canonicalized document using the message digest algorithm (e.g. SHA-256) and append it to output

In [87]:
result += sha256(str_to_bytes(canonicalized_output, 'utf-8'))
result

b'\x82\xb4\x93\x9e\xa9Mj\xf0~L)\xbe\xc5\xfd.F\x85\x1e\xa7\x97a\xafc\xbd\x10\x1b\x83\xfa\xd2u\x14\xeb\xbbF\x03B\xb9\\V\x02#,\xdbQRh\xd8\x08;b\xda\xc6\x85<,F\x8b\x95\xb3\x89u\xbc\xc5\xc2\xbbF\x03B\xb9\\V\x02#,\xdbQRh\xd8\x08;b\xda\xc6\x85<,F\x8b\x95\xb3\x89u\xbc\xc5\xc2\xbbF\x03B\xb9\\V\x02#,\xdbQRh\xd8\x08;b\xda\xc6\x85<,F\x8b\x95\xb3\x89u\xbc\xc5\xc2\xbbF\x03B\xb9\\V\x02#,\xdbQRh\xd8\x08;b\xda\xc6\x85<,F\x8b\x95\xb3\x89u\xbc\xc5\xc2\xbbF\x03B\xb9\\V\x02#,\xdbQRh\xd8\x08;b\xda\xc6\x85<,F\x8b\x95\xb3\x89u\xbc\xc5\xc2\xbbF\x03B\xb9\\V\x02#,\xdbQRh\xd8\x08;b\xda\xc6\x85<,F\x8b\x95\xb3\x89u\xbc\xc5\xc2'

## 5. Digitally sign tbs using the privateKey and the the digital proof algorithm. The resulting string is the proofValue.

**Code for sign_message can be found in [message.py](../../edit/src/message.py)**

**ISSUE:** Unsure how to handle encoding challenges. The result from Create-Verify-Hash algorithm from step 4 is 64 Bytes. However, decoding these bytes back to a string in either ASCII or UTF-8 does not work.

One potential solution would be to update the BIP322 signing function I wrote to accept bytes instead of (or aswell as) strings to be signed? This should work, I believe the reason I chose the API like this was to align it with the Bitcoin core P.R. that implements BIP322.

Are there other suggestions?

Is the real issue how do we turn a SHA256 message digest into a string? Hex?

**Question:** Is this just something we need to define as part of the BIP322 Signature Suite?

In [90]:
# For now we use hex because it works. 
# However note: as part of the sign_message, this hex string gets turned back into bytes before being signed
tbs = result.hex()

In [89]:
proofValue = sign_message(MessageSignatureFormat.SIMPLE, private_key, address, tbs)
print("\n\nBIP 322 Simple Signature as a proofValue over a VC", proofValue)



BIP 322 Simple Signature as a proofValue over a VC AkgwRQIhAMPFsLYhuQki+3zGej0N5st2qeyyFKjptFDG3ag6q1tLAiBxoyyoiBvRTneo02wOLs8qqmbFzdRnnLF4rZRgVRDcRAEhA8iCJXTXxwVa3nx91ZjdWvR+cjVLVuP6HiwsPPAl9s8x


## 6. Add a proof node to output containing a data integrity proof using the appropriate type and proofValue values as well as all of the data in the proof options (e.g. created, and if given, any additional proof options such as domain).


In [95]:


proof = options['proof']
proof['proofValue'] = proofValue

In [96]:
# Note again the type value is something we need to define. BIP322Signature2022
proof

{'type': 'Ed25519Signature2020',
 'created': '2022-08-17 13:58:30.266554',
 'verificationMethod': 'did:btcr:xyv2-xzpq-q9wa-p7t#vm-1',
 'proofPurpose': 'assertionMethod',
 'proofValue': 'AkgwRQIhAMPFsLYhuQki+3zGej0N5st2qeyyFKjptFDG3ag6q1tLAiBxoyyoiBvRTneo02wOLs8qqmbFzdRnnLF4rZRgVRDcRAEhA8iCJXTXxwVa3nx91ZjdWvR+cjVLVuP6HiwsPPAl9s8x'}

In [98]:
output['proof'] = proof

In [103]:
print("The BIP322 Signed VC \n Copy this to the verification notebook \n\n\n")
print(output)

The BIP322 Signed VC 
 Copy this to the verification notebook 



{'@context': ['https://www.w3.org/2018/credentials/v1', 'https://www.w3.org/2018/credentials/examples/v1'], 'id': 'http://example.edu/credentials/1872', 'type': ['VerifiableCredential', 'AlumniCredential'], 'issuer': 'did:btcr:xyv2-xzpq-q9wa-p7t', 'issuanceDate': '2010-01-01T19:23:24Z', 'credentialSubject': {'id': 'did:example:ebfeb1f712ebc6f1c276e12ec21', 'alumniOf': {'id': 'did:example:c276e12ec21ebfeb1f712ebc6f1', 'name': [{'value': 'Example University', 'lang': 'en'}, {'value': "Exemple d'Université", 'lang': 'fr'}]}}, 'proof': {'type': 'Ed25519Signature2020', 'created': '2022-08-17 13:58:30.266554', 'verificationMethod': 'did:btcr:xyv2-xzpq-q9wa-p7t#vm-1', 'proofPurpose': 'assertionMethod', 'proofValue': 'AkgwRQIhAMPFsLYhuQki+3zGej0N5st2qeyyFKjptFDG3ag6q1tLAiBxoyyoiBvRTneo02wOLs8qqmbFzdRnnLF4rZRgVRDcRAEhA8iCJXTXxwVa3nx91ZjdWvR+cjVLVuP6HiwsPPAl9s8x'}}
